In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
import sklearn
import xgboost as xgb

In [2]:
train = pd.read_csv('Carseats_training.csv')
test =pd.read_csv('Carseats_testing.csv')
sub = pd.read_csv('Carseats_sample_submission.csv')

In [3]:
train.head()

,ID,Sales,CompPrice,Income,Advertising,Population,Price,ShelveLoc,Age,Education,Urban,US
0,1,10.48,138,72,0,148,94,Medium,27,17,Yes,Yes
1,2,10.43,77,69,0,25,24,Medium,50,18,Yes,No
2,3,5.32,118,74,6,426,102,Medium,80,18,Yes,Yes
3,4,7.67,129,117,8,400,101,Bad,36,10,Yes,Yes
4,5,5.32,152,116,0,170,160,Medium,39,16,Yes,No


In [4]:
train['cvi'] = train['CompPrice']/train['Income']
train['cvp'] = train['CompPrice']/train['Price']
train['ivp'] = train['Income']/train['Price']
test['cvi'] = test['CompPrice']/test['Income']
test['cvp'] = test['CompPrice']/test['Price']
test['ivp'] = test['Income']/test['Price']

In [5]:
train['Urban'].value_counts()
train['ShelveLoc'].value_counts()
#train['Advertising'].value_counts()

Medium    179
Bad        76
Good       65
Name: ShelveLoc, dtype: int64

In [6]:
#leurban = preprocessing.LabelEncoder()
leshelve = preprocessing.LabelEncoder()
#leadv = preprocessing.LabelEncoder()

In [7]:
#train.loc[train.groupby('Advertising').Advertising.transform('count').lt(2), 'Advertising'] = 99
#test.loc[test.groupby('Advertising').Advertising.transform('count').lt(2), 'Advertising'] = 99


In [8]:
#leurban.fit(train['Urban'])
leshelve.fit(train['ShelveLoc'])
#leadv.fit(train['Advertising'])

LabelEncoder()

In [9]:
train['Urban'] = (train['Urban'] == 'Yes').astype(int)
train['ShelveLoc'] = leshelve.transform(train['ShelveLoc'])
#train['Advertising']=leadv.transform(train['Advertising'])
test['Urban'] = (test['Urban'] == 'Yes').astype(int)
test['ShelveLoc'] = leshelve.transform(test['ShelveLoc'])
#test['Advertising']=leadv.transform(test['Advertising'])


In [10]:
train['US'] = (train['US'] == 'Yes').astype(int)
test['US'] = (test['US'] == 'Yes').astype(int)

In [11]:
train.head()

,ID,Sales,CompPrice,Income,Advertising,Population,Price,ShelveLoc,Age,Education,Urban,US,cvi,cvp,ivp
0,1,10.48,138,72,0,148,94,2,27,17,1,1,1.916667,1.468085,0.765957
1,2,10.43,77,69,0,25,24,2,50,18,1,0,1.115942,3.208333,2.875000
2,3,5.32,118,74,6,426,102,2,80,18,1,1,1.594595,1.156863,0.725490
3,4,7.67,129,117,8,400,101,0,36,10,1,1,1.102564,1.277228,1.158416
4,5,5.32,152,116,0,170,160,2,39,16,1,0,1.310345,0.950000,0.725000


In [12]:
trainY=train['Sales']
trainX = train.drop(['ID','Sales'],axis=1)
testX= test.drop(['ID'],axis=1)

In [13]:
xgb1 = xgb.XGBRegressor(colsample_bytree=0.8,
                 gamma=0,                 
                 learning_rate=0.01,
                 max_depth=3,
                 min_child_weight=0.5,
                 n_estimators=100000,                                                                    
                 reg_alpha=0.75,
                 reg_lambda=0.45,
                 subsample=0.75,
                 seed=42) 

In [14]:
xgb1.fit(trainX,trainY,eval_metric='rmse')

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0, learning_rate=0.01, max_delta_step=0,
       max_depth=3, min_child_weight=0.5, missing=None,
       n_estimators=100000, n_jobs=1, nthread=None, objective='reg:linear',
       random_state=0, reg_alpha=0.75, reg_lambda=0.45, scale_pos_weight=1,
       seed=42, silent=True, subsample=0.75)

In [15]:
xgb1.feature_importances_

array([0.09535729, 0.07747073, 0.05911875, 0.1542119 , 0.07659023,
       0.02336449, 0.10308047, 0.04510635, 0.0124967 , 0.00705023,
       0.10562132, 0.13874668, 0.10178488], dtype=float32)

In [16]:
xgb1.score(trainX,trainY)

0.9996801759064802

In [215]:
out = xgb1.predict(trainX)

In [216]:
sub['Sales'] = pd.DataFrame(out)

In [217]:
sub.to_csv('sub.csv',index=False)